In [1]:
import numpy as np
import pandas as pd

### Node Class

Tree structure is build by Node Objects. Each node object has an attribute and a classification when trained, as well as a parent node and the value associated with the parents attribute and a classification.
The nodes are trained by tran_data and also offer the printTree function.

In [2]:
class Node:
    '''
    class Node innitialises a tree structure for a non-binary tree
    it has the typical setter and getter methods and a method to remove a 
    child from the list of children
    '''
    def __init__(self, parent = None, attribute = None, classification = None, value = None, valueIsContinuous = False, target = None):
        self.children = []  
        self.parent = parent
        self.attribute = attribute
        self.classification = classification
        self.value = value
        self.valueIsContinuous = valueIsContinuous
        self.target = target

    
    def setChild(self, node):
        self.children.append(node)
        
    def setParent(self, node):
        if (self.parent is not None):
            self.parent.children.remove(self)
        self.parent = node

    def getChildren(self):
        return self.children
    
    def getParent(self):
        return self.parent
    
    def deleteChild(self, node):
        if (node in self.children):
            node.parent = None
            self.children.remove(node)
        else:
            raise TypeError("Child not in Children")
        
    def isLeaf(self):
        if len(self.children) > 0:
            return False
        else:
            return True

    def isRoot(self):
        if (self.parent is None):
            return True
        else:
            return False
    
    def setAttribute(self, attribute):
        self.attribute = attribute
    
    def getAttribute(self):
        return self.attribute
    
    def setClassification(self, classification):
        self.classification = classification
    
    def getClassification(self):
        return self.classification
    
    def setValue(self,value):
        self.value = value

    def getValue(self):
        return self.value
    

    def printTree(self, level = 0):
        tab = "    "
        if self.isLeaf():
            print(tab*level, "classification: ", self.target, " = " , self.classification)
        else:
            for child in self.children:
                
                # printing intervals
                if child.valueIsContinuous:
                    interval = ""
                    if child.value[0] == np.NINF:
                        interval = f"smaller then {child.value[1]}"
                    elif child.value[1] == np.PINF:
                        interval = f"bigger then {child.value[0]}"
                    else:
                        interval = f"between {child.value[0]} and {child.value[1]}"
                    print(tab*level, self.attribute, ": ", interval)
                
                # printing discrete values
                else:
                    print(tab*level, self.attribute, ": ", child.value)
                
                # traverse deeper into the tree
                child.printTree(level + 1)

### train_Data Class

The class represents a data (sub)set that is used to train the (sub)tree. It has a (root) node which it is responsible to train (choose attribute, classification, and child nodes) via the id3 function.
I can also deal with continuous variables by ...

In [3]:
class train_data:
    '''
    class train_data has all important functions for ID3 Algorithm:
    it can calculate the entropy of some data, the information gain, choose an attriute.
    within the ID3 algorithm a tree will be trained.
    the function retrain(data) retrains a trained tree with new data.
    '''
    def __init__(self, data, target, attributes, node:Node = None, recursion_depth = None, continuous_splitting = 0.1,  max_recursion = 10):
        
        self.data = data
        if not isinstance(self.data, pd.DataFrame):
            raise TypeError("Data has to be a Pandas Dataframe")
        
        self.target = target
        if not isinstance(self.target, str):
            raise TypeError("Taget has to be of type string")
        
        self.attributes = attributes
        if not isinstance(attributes, list):
            raise TypeError("Attributes have to have structure list")
            
        for attribute in self.attributes:
            if not isinstance(attribute, str):
                raise TypeError("Attributes have to be of type string")

        self.node = node
        self.continuous_splitting = continuous_splitting
        self.recursion_depth = recursion_depth
        self.max_recursion = max_recursion
    
    ######################################
    ## methods for continuous variables ##
    ######################################
    
    def is_continuous(self, values):
        # checks is variable is a continuous variable
        # (it is continuous if it has more than 10 different values and is a numericla scalar)
        if len(values) > 10:
            if isinstance(list(values)[5], int) or isinstance(list(values)[5], float):
                return True
        return False

    
    def getBoundaries(self, tColumn, aColumn):
        # by looking at the target column and the attribute column the
        # function decides on decision boundaries in a continuous varibale, where classification changes
        # aColumn -> attribute column with the continuous values
        # tColumn -> target column with the classification
        
        # 1) sort the two columns by attribute values
        columns = pd.DataFrame(data={"a":list(aColumn), "t":list(tColumn)}).sort_values(by="a")
        columns.index = range(len(columns))

        # 2) find decision boundaries where classification changes
        leftBound = np.NINF # first interval has negative infinity as left boundary
        rightBound = None
        boundaries = []
        currentClass = columns["t"][0]
        
        for i in range(len(columns)):
            
            # when classification changes
            if(columns["t"][i] != currentClass):
                currentClass = columns["t"][i]
                
                # get the value in the middel of the values where classification changes
                beforeSwitch = columns["a"][i-1]
                afterSwitch = columns["a"][i]
                rightBound = (beforeSwitch + afterSwitch) / 2

                # safe the tupple of two boundaries 
                # represents an interval with a uniform classification
                boundaries.append((leftBound, rightBound))
                leftBound = rightBound
        
        # last interval has negative infinity as right boundary
        boundaries.append((leftBound, np.PINF))
        
        # if the getBoundaries function returns more then 10 intervals
        # set intervals indipendent of classification
        if len(boundaries) > 10:
            return self.setBoundaries(aColumn)
        
        return boundaries
    
    def setBoundaries(self, aColumn):
        # if the getBoundaries function returns more then 10 intervals
        # sets 10 eaqually sized intervals indipendent of classification
        
        # calculate size of intervals
        maximum = np.max(aColumn)
        minimum = np.min(aColumn)
        stepsize = (maximum - minimum)/ 10
        boundaries = []
        
        # make a tupel for each interval
        leftBound = np.NINF
        rightBound = minimum + stepsize
        for i in range(9):
            boundaries.append((leftBound, rightBound))
            leftBound = rightBound
            rightBound = leftBound + stepsize
        boundaries.append((leftBound, np.PINF))
        
        return boundaries
        
    
    def replaceContinuous(self, boundaries, aColumn):
        # replaces the continuous values of an attribute by the
        # tuples that represent an interval
        
        newAColumn = []
        for value in aColumn:
            for l, r in boundaries:
                if value >= l and value < r:
                    newAColumn.append((l, r))
        return newAColumn
    
    
    #######################################
    ## methods for choosing an attribute ##
    #######################################
    
    
    def entropy(self):
        # calculates entropy 
        targetColumn = self.data.loc[:, self.target]

        values = set(targetColumn)
        entropySum = 0
        for value in values:
            p = list(targetColumn).count(value) / len(targetColumn)
            entropySum = entropySum + (- p * np.log(p))

        return entropySum
    

    def informationGain(self, attributeColumn, values):
        # calculates the informationGain
        
        gainSum = 0
        for value in values:
            
            mask = lambda aColumn, value :(row == value for row in aColumn) 
            subsetData = self.data.iloc[mask(attributeColumn, value),:]
            subset = train_data(subsetData, self.target, self.attributes)
            # claculate entropy and normalize by size of subsets
            gainSum = gainSum + (subsetData.shape[0] / self.data.shape[0]) * subset.entropy()

        # substract summed and weighted entropy of subsets from entropy of whole set    
        infoGain = self.entropy() - gainSum

        return infoGain

    def gainRatio(self, attributeColumn, values):
        # calculating the Gain Ratio instead of the InforamtionGain
        # to prefer attributes with few values
        
        infoGain = self.informationGain(attributeColumn, values)

        splitInfo = 0.0
        for value in values:
            
            subset = attributeColumn[attributeColumn == value]
            # proportion of subset size and whole set size
            s = len(subset) / len(attributeColumn)
            if s != 0.0:
                splitInfo = splitInfo + ((- s) * np.log(s))
        
        # to avoid dividing by zero
        if splitInfo == 0:
            splitInfo = infoGain
            if infoGain == 0:
                return 0
            
        return infoGain / splitInfo
        

    def chooseAttribute(self):
        # chooses an attribute that maximises GainRatio
        
        maxGain= 0
        maxAttribute = ""

        # calculate Gain Ratio for each attribute
        for attribute in self.attributes:
            
            attributeColumn = self.data.loc[:, attribute]
            values = set(attributeColumn)
            gain = 0

            # replace the values in attributeColumn with continuous values by Intervals
            if self.is_continuous(values):
                targetColumn = self.data[self.target]
                boundaries = self.getBoundaries(targetColumn, attributeColumn)
                attributeColumn = self.replaceContinuous(boundaries, attributeColumn)
                values = set(attributeColumn)            
                
            # calculate gainRatio
            gain = self.gainRatio(attributeColumn, values)

            # store attribute with highest information gain
            if gain >= maxGain:
                maxGain = gain
                maxAttribute = attribute

        # choose attribute with highest Information Gain
        return maxAttribute

    ############################################
    ## methods for building the decision tree ##
    ############################################
    
    def classify(self):
        # returns the most commen classification of the dataset
        
        targetColumn = self.data.loc[:, self.target]
        values = set(targetColumn)
        maxClass = 0  # highest number of values
        classification = "" # classification of most common value
        for value in values:
            # check if calssification value is more common then other classification values
            if list(targetColumn).count(value) > maxClass:
                maxClass = list(targetColumn).count(value)
                classification = value

        return classification

    def id3(self):
        # base cases:
        # 1) all instances have same target value -> leaf node with target value
        if (self.data[self.target].nunique() == 1):
            self.node.setClassification(self.data[self.target].iloc[0])
            print("basecase1")
            return 
        # 2) out of discriptive features -> leaf node with majority of target values
        if (not self.attributes):
            self.node.setClassification(self.classify())
            print("basecase2")
            return
        # 3) no instances left in dataset -> take majority of parent node
        if (self.data is None):
            parent = self.node.getParent()
            self.node.setClassification(parent.getClassification())
            print("basecase3")
            return
        # 4) maximal recursion depth:
        if self.recursion_depth == self.max_recursion:
            self.node.setClassification(self.classify())
            print("basecase4")
            return


        # recursive case:
        # choose attribute with highest explainatory power
        print("in recursion")
        print("attributs: ", self.attributes)
        attribute = self.chooseAttribute()
        self.node.setAttribute(attribute)
        self.node.setClassification(self.classify())

        # split data according to attribute
        attributeColumn = self.data.loc[:, attribute]
        values = set(attributeColumn)
        new_attributes = self.attributes
        new_attributes.remove(attribute)
        recursion_depth = self.recursion_depth + 1

        valueIsContinuous=False
        
        # chosen attribute is a continuous variable:
        if self.is_continuous(values):
            print("continuous")
            
            targetColumn = self.data[self.target]
            boundaries = self.getBoundaries(targetColumn, attributeColumn)
            attributeColumn = self.replaceContinuous(boundaries, attributeColumn)
            values = set(attributeColumn)
            valueIsContinuous=True
        
        # create leaf node for each attribute value
        for value in values:
            # get the subset determined by the attribute value
            mask = lambda aColumn, value :(row == value for row in aColumn) 
            subsetData = self.data.iloc[mask(attributeColumn, value),:]
            # create a node in the tree
            childNode = Node(parent=self.node, value=value, valueIsContinuous=valueIsContinuous, target=self.target)
            self.node.setChild(childNode)
            # train the node with the data subset
            subset = train_data(data=subsetData, target=self.target, attributes=new_attributes, node=childNode, recursion_depth=recursion_depth)
            subset.id3() # recursive call on all partitions
            

    def retrain(self, data):
        self.data = data
        self.id3()
    

### test_data Class

to classify datapoints and test accuracy of tree

In [4]:
class test_data:
    
    def __init__(self, testData, target, node:Node):
        self.testData = testData
        self.target = target
        self.node = node

        # check whether node is trained:
        if node.getAttribute() is None:
            raise TypeError("node has to be part of a trained Decisiontree")


    def calcError(self, datapoint):

        # compare leaf node classification and datapoint classification (basecase)
        if self.node.isLeaf() == False:
            return self.node.getClassification() == datapoint[self.target]

        # traverse down the tree with the decision nodes (recursive case)
        else:
            attribute = self.node.getAttribute()
            dataValue = datapoint[attribute]
            for child in self.node.children:
                # for interval values
                if child.valueIsContinuous:
                    if dataValue >= child.getValue()[0] and dataValue < child.getValue()[1]:
                        return test_data(datapoint, self.target, child).calcError()
                # for discrete values
                if child.getValue() is dataValue:
                    return test_data(datapoint, self.target, child).calcError()
        
        # if there are no children with the right value at decision node, use current classification (base case)
        return self.node.getClassification() == self.target

    
    def calcClassification(self, datapoint):
        
        # get leaf node classification (basecase)
        if self.node.isLeaf() == False:
            return self.node.getClassification()

        # traverse down the tree with the decision nodes (recursive case)
        else:
            attribute = self.node.getAttribute()
            dataValue = datapoint[attribute]
            for child in self.node.children:
                # for interval values
                if child.valueIsContinuous:
                    if dataValue >= child.getValue()[0] and dataValue < child.getValue()[1]:
                        return test_data(datapoint, self.target, child).calcClassification()
                # for discrete values
                if child.getValue() is dataValue:
                    return test_data(datapoint, self.target, child).calcClassification()
        
        # if there are no children with the right value at decision node, get current classification (base case)
        return self.node.getClassification()
        
    def classify(self):
        classificationArray = []
        for i in range(self.testData.shape[0]):
            datapoint = self.testData.loc[i]
            classificationArray.append(self.calcClassification(datapoint))
        
        return classificationArray
    
    def accuracy(self):
        errorArray = []
        for i in range(self.testData.shape[0]):
            datapoint = self.testData.loc[i]
            errorArray.append(self.calcError(datapoint))
        
        return np.mean(errorArray)



### Main

In [5]:
d= {"gender": ["f", "f", "f", "f", "f", "m", "m", "m", "m", "m"],
                   "vegan": [True, True, True, False, False, True, False, False, False, False],
                   "coxi": [True, True, True, False, True, True, True, False, False, False],
                   "green": [True, True, True, False, False, True, False, True, False, True]}

data = pd.DataFrame(data = d)

target = "vegan"
attributes = list(data.columns)
attributes.remove(target)

rootNode = Node()

decisionTree = train_data(data=data, target=target, attributes=attributes, node=rootNode, recursion_depth=5)
decisionTree.id3()

rootNode.printTree()

in recursion
attributs:  ['gender', 'coxi', 'green']
basecase1
in recursion
attributs:  ['gender', 'coxi']
basecase1
basecase1
 green :  False
     classification:  vegan  =  False
 green :  True
     coxi :  False
         classification:  vegan  =  False
     coxi :  True
         classification:  vegan  =  True


In [6]:
datapoints = {"gender": ["f", "m", "m"],
                "vegan": [True, False, False],
                "coxi": [False, False, False],
                "green": [False, False, True]}

datapoints = pd.DataFrame(data = datapoints)

print("accuracy: ", test_data(datapoints, "vegan", rootNode).accuracy() )
print("classifications: ", test_data(datapoints, "vegan", rootNode).classify())

accuracy:  0.6666666666666666
classifications:  [False, False, False]


In [7]:
# 1. load data
data = pd.read_csv("data/pokemon_no_duplicates.csv")

# 2. prepare data
#data = prepare_data(data)

# choose the target value
data = data.drop(columns=["Name", "#"])
target = "Generation"
attributes = list(data.columns)
attributes.remove(target)
# 3. split_data with id3
rootNode = Node()
decisionTree = train_data(data=data, target=target, attributes=attributes, node=rootNode, recursion_depth=0)
decisionTree.id3()

rootNode.printTree()

in recursion
attributs:  ['Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Legendary']
in recursion
attributs:  ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Legendary']
continuous
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Def', 'Speed', 'Legendary']
basecase1
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Def', 'Legendary']
basecase1
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Legendary']
basecase1
basecase1
basecase1
basecase1
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'Attack', 'Defense', 'Legendary']
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'Attack', 'Legendary']
basecase1
basecase1
basecase1
basecase1
in recursion
attributs:  ['Type 1', 'Total', 'Legendary']
basecase1
basecase1
basecase1
basecase1
in recursion
att

In [8]:

#datapoints = data.loc(5) #error

#print("accuracy: ", test_data(datapoints, "Generation", rootNode).accuracy() )
#print("classifications: ", test_data(datapoints, "Generation", rootNode).classify())